# CNN for many subjects

$$ x = y^2 + 2$$

In [1]:
%pylab
%matplotlib inline

import glob
import os
import mne
CORPORA_PATH = "~/corpora/sets"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))

def normalize_subject(X):
    mean = X.mean(axis=(0, 2)).reshape(-1, 1)
    std = X.std(axis=(0, 2)).reshape(-1, 1)
    return (X - mean) / std

def load_data(filename, normalize=True):
    data_mne = mne.io.read_raw_eeglab(filename, preload=True, event_id={"0": 1, "1": 2})
    data_mne.filter(0, 20)
    events = mne.find_events(data_mne)
    epochs = mne.Epochs(
        data_mne, events,
        baseline=(None, 0), tmin=-0.1, tmax=0.7)

    epochs.load_data()
    
    ch_names = epochs.ch_names
    
    X = epochs.get_data()[:, :-1]
    y = (events[:, 2] == 2).astype('float')

    if len(events) != len(epochs):
        raise ValueError("Epochs events mismatch")
    if normalize: 
        X = normalize_subject(X)
    
    
    return X, y 


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


Targets appear as 2 in the third column


We remove last channel as well

# CNN with more data

In [2]:
%%capture 

filenames = files


X = None
y = None
print(filenames)
for filename in filenames:
    try:
        X_subject, y_subject = load_data(filename)

        if X is None:
            X, y = X_subject, y_subject
        else:
            X = np.vstack((X, X_subject))
            y = np.vstack((y.reshape(-1,1), y_subject.reshape(-1,1)))
    except ValueError as e:
        print(e)

In [3]:
X.shape

(287640, 14, 104)

In [4]:
X_t = X[:, :, :, np.newaxis]

X_t.shape

(287640, 14, 104, 1)

In [5]:
 sum(y) / len(y)

0.16666666666666666

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_t, y, test_size=0.1, stratify=y)

X_train.shape, y_train.shape

((258876, 14, 104, 1), (258876, 1))

In [13]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout

model = Sequential()

n_kernels = 10
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy


In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.2.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)


model.fit(
    X_train, y_train, epochs=50, 
    batch_size=256, class_weight={0:1, 1:6}, validation_split=0.1,
    callbacks=[checkpointer, early_stopping]
)

Train on 232988 samples, validate on 25888 samples
Epoch 1/50
232988/232988 [==============================] - 90s 384us/step - loss: 1.2976 - acc: 0.4104 - val_loss: 1.2316 - val_acc: 0.5265
Epoch 2/50
232988/232988 [==============================] - 89s 382us/step - loss: 1.2317 - acc: 0.4569 - val_loss: 1.2292 - val_acc: 0.4601
Epoch 3/50
232988/232988 [==============================] - 89s 382us/step - loss: 1.2264 - acc: 0.4603 - val_loss: 1.2274 - val_acc: 0.4553
Epoch 4/50
232988/232988 [==============================] - 89s 382us/step - loss: 1.2233 - acc: 0.4653 - val_loss: 1.2207 - val_acc: 0.4867
Epoch 5/50
232988/232988 [==============================] - 88s 379us/step - loss: 1.2227 - acc: 0.4625 - val_loss: 1.2213 - val_acc: 0.5183
Epoch 6/50
232988/232988 [==============================] - 88s 379us/step - loss: 1.2216 - acc: 0.4651 - val_loss: 1.2229 - val_acc: 0.5160
Epoch 7/50
232988/232988 [==============================] - 88s 379us/step - loss: 1.2211 - acc: 0.4618

In [17]:
from keras.models import load_model

model = load_model("model.2.h5")

y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.43391044361006814
Precision  = 0.19811866099111883
Recall     = 0.7863996662494785
ROC AUC    = 0.6359560579217973



## Model with two layers

In [18]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout, MaxPool2D

model = Sequential()

n_kernels = 10
model.add(Conv2D(2*n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(MaxPool2D((1, 4)))
model.add(Dropout(0.35))
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(2*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(MaxPool2D((1, 4)))

model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', 
              metrics=['accuracy']) # reporting the accuracy

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.2conv_with_maxpool.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=256, class_weight={0:1, 1:6}, validation_split=0.1,
    callbacks=[checkpointer, early_stopping]
)

Train on 232988 samples, validate on 25888 samples
Epoch 1/30
232988/232988 [==============================] - 88s 377us/step - loss: 1.2404 - acc: 0.3876 - val_loss: 1.2312 - val_acc: 0.4926
Epoch 2/30
232988/232988 [==============================] - 86s 371us/step - loss: 1.2240 - acc: 0.4836 - val_loss: 1.2210 - val_acc: 0.4896
Epoch 3/30
232988/232988 [==============================] - 87s 372us/step - loss: 1.2185 - acc: 0.4927 - val_loss: 1.2187 - val_acc: 0.5652
Epoch 4/30
232988/232988 [==============================] - 86s 371us/step - loss: 1.2140 - acc: 0.5055 - val_loss: 1.2155 - val_acc: 0.5521
Epoch 5/30
232988/232988 [==============================] - 86s 371us/step - loss: 1.2111 - acc: 0.5070 - val_loss: 1.2128 - val_acc: 0.5290
Epoch 6/30
232988/232988 [==============================] - 86s 370us/step - loss: 1.2082 - acc: 0.5031 - val_loss: 1.2091 - val_acc: 0.5268
Epoch 7/30
232988/232988 [==============================] - 86s 370us/step - loss: 1.2067 - acc: 0.5116

In [21]:
from keras.models import load_model

model = load_model("model.2conv_with_maxpool.h5")

y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.5206160478375748
Precision  = 0.21790127328608166
Recall     = 0.7246558197747184
ROC AUC    = 0.6565931740221097



## Conv with two layers but one simple

In [28]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout, MaxPool2D

model = Sequential()

n_kernels = 10
model.add(Conv2D(n_kernels, (14, 14), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(MaxPool2D((1, 4)))

model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.50))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy

In [29]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.3conv_with_maxpool.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

model.fit(
    X_train, y_train, epochs=10, 
    batch_size=256, class_weight={0:1, 1:6}, validation_split=0.1,
    callbacks=[checkpointer, early_stopping]
)

Train on 232988 samples, validate on 25888 samples
Epoch 1/10
232988/232988 [==============================] - 22s 93us/step - loss: 1.2591 - acc: 0.4142 - val_loss: 1.2371 - val_acc: 0.4066
Epoch 2/10
232988/232988 [==============================] - 21s 91us/step - loss: 1.2445 - acc: 0.4198 - val_loss: 1.2299 - val_acc: 0.5048
Epoch 3/10
232988/232988 [==============================] - 21s 91us/step - loss: 1.2358 - acc: 0.4199 - val_loss: 1.2266 - val_acc: 0.3874
Epoch 4/10
232988/232988 [==============================] - 21s 91us/step - loss: 1.2317 - acc: 0.4206 - val_loss: 1.2259 - val_acc: 0.5162
Epoch 5/10
232988/232988 [==============================] - 21s 91us/step - loss: 1.2306 - acc: 0.4144 - val_loss: 1.2207 - val_acc: 0.4313
Epoch 6/10
232988/232988 [==============================] - 21s 90us/step - loss: 1.2298 - acc: 0.4214 - val_loss: 1.2228 - val_acc: 0.4159
Epoch 7/10
232988/232988 [==============================] - 21s 89us/step - loss: 1.2296 - acc: 0.4279 - val_

In [30]:
from keras.models import load_model


y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.4183354192740926
Precision  = 0.1965838035687052
Recall     = 0.8066332916145181
ROC AUC    = 0.6368764216291085

